In [230]:
import os
import pandas as pd

In [231]:
results = os.listdir('search/')
#results

In [232]:
open_file = pd.read_csv('search/'+results[0])
open_file['dates'] = str(results[0])
weekends = open_file.head(0)

In [233]:
for file in results: 
    open_file = pd.read_csv('search/'+file)
    open_file['dates'] = str(file)
    weekends = weekends.append(open_file)

In [234]:
weekends.reset_index(drop = True)
weekends.head(1)

,name,nightly cost,category,superhost,url,guests_rental,total cost,reviews,dates
0,NC5 The Joint!! Downtown Nashville Studio!,Price:$55 / night,Entire apartment in Nashville,NaN,/rooms/46779266?check_in=2020-12-17&check_out=...,6 guests · Studio · 2 beds · 1 bath,Price:$245 totalShow details,Price:$55 / nightPrice:$245 totalShow details,ci2020-12-17co2020-12-20.csv


In [235]:
# new data frame with split value columns 
dates = weekends['dates'].str.split('co', n = 1, expand = True)
weekends['checkin'] = dates[0] 
weekends['checkout'] = dates[1]
#weekends.head()

In [236]:
weekends['checkin'] = weekends['checkin'].str.replace('ci','')
weekends['checkout'] = weekends['checkout'].str.replace('.csv','')
weekends.drop('dates', axis=1, inplace=True)
#weekends.head()                                                    

In [237]:
weekends['guests'] = weekends['guests_rental'].str.partition(' · ')[0]
#weekends.head()

In [238]:
weekends['guest_rental2'] = weekends['guests_rental'].str.partition(' · ')[2]
#weekends.head()

In [239]:
weekends['bedroom'] = weekends['guest_rental2'].str.partition(' · ')[0] 
#weekends.head()

In [240]:
weekends['guest_rental3'] = weekends['guest_rental2'].str.partition(' · ')[2]
#weekends.head()

In [241]:
weekends['beds'] = weekends['guest_rental3'].str.partition(' · ')[0] 
#weekends.head()

In [242]:
weekends['bath'] = weekends['guest_rental3'].str.partition(' · ')[2]
#weekends.head()

In [243]:
weekends.drop(columns = ['guests_rental', 'guest_rental2', 'guest_rental3'], axis=1, inplace=True)
#weekends.head()

In [244]:
#weekends.to_csv('datacleaning.csv')

In [245]:
weekends['not_discounted'] = weekends['nightly cost'].str.partition('Previous price:')[0]
#weekends.head()

In [246]:
weekends['test'] = weekends['nightly cost'].str.partition('Previous price:')[2]
#weekends.head()

In [247]:
weekends['discounted'] = weekends['test'].str.partition(':')[0]
#weekends.head()

In [248]:
weekends['discounted'] = weekends['test'].str.partition(':')[2]
#weekends.head()

In [249]:
weekends['previous_price'] = weekends['test'].str.partition(':')[0]
#weekends.head()

In [250]:
weekends.drop(columns=['test', 'nightly cost'], axis=1, inplace=True)
#weekends.head()

In [251]:
weekends['not_discounted'] = weekends['not_discounted'].str.replace('Price:', '') 
weekends['not_discounted'] = weekends['not_discounted'].str.replace(' / night', '')

#weekends.head()

In [252]:
weekends['discounted'] = weekends['discounted'].str.replace(' / night', '')
#weekends.head()

In [253]:
weekends['previous_price'] = weekends['previous_price'].str.replace('Discounted price', '')
#weekends.head()

In [254]:
weekends['current_nightly'] = weekends['not_discounted'] + weekends['discounted']
#weekends.head()

In [255]:
weekends['guests'] = weekends['guests'].str.replace(' guests', '')
#weekends.head()

In [256]:
weekends['guests'] = weekends['guests'].str.replace(' guest', '')
#weekends.head(25)

In [257]:
weekends['beds'] = weekends['beds'].str.replace(' beds', '')
weekends['beds'] = weekends['beds'].str.replace(' bed', '')
#weekends.head()

In [258]:
weekends['total cost'] = weekends['total cost'].str.replace('Price:', '')
weekends['total cost'] = weekends['total cost'].str.replace('totalShow details', '')
#weekends.head()

In [259]:
weekends['review_test'] = weekends['reviews'].str.partition('Rating')[2]
#weekends.head()

In [260]:
weekends['avg_stars'] = weekends['review_test'].str.partition(' out of')[0]
#weekends.head()

In [261]:
weekends['review_count2'] = weekends['review_test'].str.partition('(')[2]
#weekends.head()

In [262]:
weekends['review_count'] = weekends['review_count2'].str.partition(')')[0]
#weekends.head()

In [263]:
weekends.drop(columns = ['review_count2', 'review_test', 'reviews'], axis=1, inplace=True)
#weekends.head()

In [264]:
weekends = weekends[['name', 'category', 'superhost', 'url', 'avg_stars', 'review_count', 'guests','beds', 'bedroom', 'bath', 'checkin', 'checkout', 'previous_price', 'discounted', 'not_discounted', 'current_nightly', 'total cost']]
weekends.head()

,name,category,superhost,url,avg_stars,review_count,guests,beds,bedroom,bath,checkin,checkout,previous_price,discounted,not_discounted,current_nightly,total cost
0,NC5 The Joint!! Downtown Nashville Studio!,Entire apartment in Nashville,NaN,/rooms/46779266?check_in=2020-12-17&check_out=...,,,6,2,Studio,1 bath,2020-12-17,2020-12-20,,,$55,$55,$245
1,"Brand-New Condo, 2 Blocks to Centennial Park",Entire apartment in Nashville,NaN,/rooms/35440277?check_in=2020-12-17&check_out=...,4.64,14,4,2,1 bedroom,1 bath,2020-12-17,2020-12-20,$122,$58,,$58,$329
2,DOWNTOWN NASHVILLE - COMFY & COZY (C),Entire apartment in Nashville,SUPERHOST,/rooms/29543248?check_in=2020-12-17&check_out=...,4.93,76,6,2,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,$63,$63,$302
3,Tiny haven,Entire apartment in Nashville,SUPERHOST,/rooms/20137844?check_in=2020-12-17&check_out=...,4.93,193,4,4,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,$54,$54,$259
4,1 BR Apartment!! Just 2 miles to Downtown!,Entire apartment in Nashville,NaN,/rooms/44064578?check_in=2020-12-17&check_out=...,4.0,10,4,1,1 bedroom,1 bath,2020-12-17,2020-12-20,$87,$62,,$62,$257


In [268]:
weekends.dtypes

name               object
category           object
superhost          object
url                object
avg_stars          object
review_count       object
guests             object
beds               object
bedroom            object
bath               object
checkin            object
checkout           object
previous_price     object
discounted         object
not_discounted     object
current_nightly    object
total cost         object
dtype: object

In [269]:
#weekends['current_nightly'] = weekends['current_nightly'].astype(str)
#weekends.head()

In [270]:
weekends['current_nightly'] = weekends['current_nightly'].astype(str)

weekends['current_nightly'] = weekends['current_nightly'].str.replace('$','')
weekends['current_nightly'] = weekends['current_nightly'].str.replace(',','')
weekends['current_nightly'] = weekends['current_nightly'].str.replace('nan', '0')

weekends['current_nightly'] = weekends['current_nightly'].astype(int)

weekends.head()

,name,category,superhost,url,avg_stars,review_count,guests,beds,bedroom,bath,checkin,checkout,previous_price,discounted,not_discounted,current_nightly,total cost
0,NC5 The Joint!! Downtown Nashville Studio!,Entire apartment in Nashville,NaN,/rooms/46779266?check_in=2020-12-17&check_out=...,,,6,2,Studio,1 bath,2020-12-17,2020-12-20,,,$55,55,$245
1,"Brand-New Condo, 2 Blocks to Centennial Park",Entire apartment in Nashville,NaN,/rooms/35440277?check_in=2020-12-17&check_out=...,4.64,14,4,2,1 bedroom,1 bath,2020-12-17,2020-12-20,$122,$58,,58,$329
2,DOWNTOWN NASHVILLE - COMFY & COZY (C),Entire apartment in Nashville,SUPERHOST,/rooms/29543248?check_in=2020-12-17&check_out=...,4.93,76,6,2,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,$63,63,$302
3,Tiny haven,Entire apartment in Nashville,SUPERHOST,/rooms/20137844?check_in=2020-12-17&check_out=...,4.93,193,4,4,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,$54,54,$259
4,1 BR Apartment!! Just 2 miles to Downtown!,Entire apartment in Nashville,NaN,/rooms/44064578?check_in=2020-12-17&check_out=...,4.0,10,4,1,1 bedroom,1 bath,2020-12-17,2020-12-20,$87,$62,,62,$257


In [271]:
weekends['room_fee'] = weekends['current_nightly'] * 3 
weekends.head()

,name,category,superhost,url,avg_stars,review_count,guests,beds,bedroom,bath,checkin,checkout,previous_price,discounted,not_discounted,current_nightly,total cost,room_fee
0,NC5 The Joint!! Downtown Nashville Studio!,Entire apartment in Nashville,NaN,/rooms/46779266?check_in=2020-12-17&check_out=...,,,6,2,Studio,1 bath,2020-12-17,2020-12-20,,,$55,55,$245,165
1,"Brand-New Condo, 2 Blocks to Centennial Park",Entire apartment in Nashville,NaN,/rooms/35440277?check_in=2020-12-17&check_out=...,4.64,14,4,2,1 bedroom,1 bath,2020-12-17,2020-12-20,$122,$58,,58,$329,174
2,DOWNTOWN NASHVILLE - COMFY & COZY (C),Entire apartment in Nashville,SUPERHOST,/rooms/29543248?check_in=2020-12-17&check_out=...,4.93,76,6,2,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,$63,63,$302,189
3,Tiny haven,Entire apartment in Nashville,SUPERHOST,/rooms/20137844?check_in=2020-12-17&check_out=...,4.93,193,4,4,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,$54,54,$259,162
4,1 BR Apartment!! Just 2 miles to Downtown!,Entire apartment in Nashville,NaN,/rooms/44064578?check_in=2020-12-17&check_out=...,4.0,10,4,1,1 bedroom,1 bath,2020-12-17,2020-12-20,$87,$62,,62,$257,186


In [273]:
weekends['total cost'] = weekends['total cost'].astype(str)

weekends['total cost'] = weekends['total cost'].str.replace('$','')
weekends['total cost'] = weekends['total cost'].str.replace(',','')
weekends['total cost'] = weekends['total cost'].str.replace(' total','')
weekends['total cost'] = weekends['total cost'].str.replace('nan', '0')

weekends['total cost'] = weekends['total cost'].astype(int)

weekends.head()

,name,category,superhost,url,avg_stars,review_count,guests,beds,bedroom,bath,checkin,checkout,previous_price,discounted,not_discounted,current_nightly,total cost,room_fee
0,NC5 The Joint!! Downtown Nashville Studio!,Entire apartment in Nashville,NaN,/rooms/46779266?check_in=2020-12-17&check_out=...,,,6,2,Studio,1 bath,2020-12-17,2020-12-20,,,$55,55,245,165
1,"Brand-New Condo, 2 Blocks to Centennial Park",Entire apartment in Nashville,NaN,/rooms/35440277?check_in=2020-12-17&check_out=...,4.64,14,4,2,1 bedroom,1 bath,2020-12-17,2020-12-20,$122,$58,,58,329,174
2,DOWNTOWN NASHVILLE - COMFY & COZY (C),Entire apartment in Nashville,SUPERHOST,/rooms/29543248?check_in=2020-12-17&check_out=...,4.93,76,6,2,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,$63,63,302,189
3,Tiny haven,Entire apartment in Nashville,SUPERHOST,/rooms/20137844?check_in=2020-12-17&check_out=...,4.93,193,4,4,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,$54,54,259,162
4,1 BR Apartment!! Just 2 miles to Downtown!,Entire apartment in Nashville,NaN,/rooms/44064578?check_in=2020-12-17&check_out=...,4.0,10,4,1,1 bedroom,1 bath,2020-12-17,2020-12-20,$87,$62,,62,257,186


In [274]:
weekends['taxes_fees_cleaning'] = weekends['total cost'] - weekends['room_fee']
weekends.head()

,name,category,superhost,url,avg_stars,review_count,guests,beds,bedroom,bath,checkin,checkout,previous_price,discounted,not_discounted,current_nightly,total cost,room_fee,taxes_fees_cleaning
0,NC5 The Joint!! Downtown Nashville Studio!,Entire apartment in Nashville,NaN,/rooms/46779266?check_in=2020-12-17&check_out=...,,,6,2,Studio,1 bath,2020-12-17,2020-12-20,,,$55,55,245,165,80
1,"Brand-New Condo, 2 Blocks to Centennial Park",Entire apartment in Nashville,NaN,/rooms/35440277?check_in=2020-12-17&check_out=...,4.64,14,4,2,1 bedroom,1 bath,2020-12-17,2020-12-20,$122,$58,,58,329,174,155
2,DOWNTOWN NASHVILLE - COMFY & COZY (C),Entire apartment in Nashville,SUPERHOST,/rooms/29543248?check_in=2020-12-17&check_out=...,4.93,76,6,2,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,$63,63,302,189,113
3,Tiny haven,Entire apartment in Nashville,SUPERHOST,/rooms/20137844?check_in=2020-12-17&check_out=...,4.93,193,4,4,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,$54,54,259,162,97
4,1 BR Apartment!! Just 2 miles to Downtown!,Entire apartment in Nashville,NaN,/rooms/44064578?check_in=2020-12-17&check_out=...,4.0,10,4,1,1 bedroom,1 bath,2020-12-17,2020-12-20,$87,$62,,62,257,186,71


In [279]:
weekends['previous_price'] = weekends['previous_price'].astype(str)

weekends['previous_price'] = weekends['previous_price'].str.replace('$','')
weekends['previous_price'] = weekends['previous_price'].str.replace(',','')
weekends['previous_price'] = weekends['previous_price'].str.replace(' total','')
weekends['previous_price'] = weekends['previous_price'].str.replace('nan', '0')

weekends.head()

,name,category,superhost,url,avg_stars,review_count,guests,beds,bedroom,bath,checkin,checkout,previous_price,discounted,not_discounted,current_nightly,total cost,room_fee,taxes_fees_cleaning
0,NC5 The Joint!! Downtown Nashville Studio!,Entire apartment in Nashville,NaN,/rooms/46779266?check_in=2020-12-17&check_out=...,,,6,2,Studio,1 bath,2020-12-17,2020-12-20,,,$55,55,245,165,80
1,"Brand-New Condo, 2 Blocks to Centennial Park",Entire apartment in Nashville,NaN,/rooms/35440277?check_in=2020-12-17&check_out=...,4.64,14,4,2,1 bedroom,1 bath,2020-12-17,2020-12-20,122,$58,,58,329,174,155
2,DOWNTOWN NASHVILLE - COMFY & COZY (C),Entire apartment in Nashville,SUPERHOST,/rooms/29543248?check_in=2020-12-17&check_out=...,4.93,76,6,2,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,$63,63,302,189,113
3,Tiny haven,Entire apartment in Nashville,SUPERHOST,/rooms/20137844?check_in=2020-12-17&check_out=...,4.93,193,4,4,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,$54,54,259,162,97
4,1 BR Apartment!! Just 2 miles to Downtown!,Entire apartment in Nashville,NaN,/rooms/44064578?check_in=2020-12-17&check_out=...,4.0,10,4,1,1 bedroom,1 bath,2020-12-17,2020-12-20,87,$62,,62,257,186,71


In [280]:
weekends['discounted'] = weekends['discounted'].astype(str)

weekends['discounted'] = weekends['discounted'].str.replace('$','')
weekends['discounted'] = weekends['discounted'].str.replace(',','')
weekends['discounted'] = weekends['discounted'].str.replace(' total','')
weekends['discounted'] = weekends['discounted'].str.replace('nan', '0')

weekends.head()

,name,category,superhost,url,avg_stars,review_count,guests,beds,bedroom,bath,checkin,checkout,previous_price,discounted,not_discounted,current_nightly,total cost,room_fee,taxes_fees_cleaning
0,NC5 The Joint!! Downtown Nashville Studio!,Entire apartment in Nashville,NaN,/rooms/46779266?check_in=2020-12-17&check_out=...,,,6,2,Studio,1 bath,2020-12-17,2020-12-20,,,$55,55,245,165,80
1,"Brand-New Condo, 2 Blocks to Centennial Park",Entire apartment in Nashville,NaN,/rooms/35440277?check_in=2020-12-17&check_out=...,4.64,14,4,2,1 bedroom,1 bath,2020-12-17,2020-12-20,122,58,,58,329,174,155
2,DOWNTOWN NASHVILLE - COMFY & COZY (C),Entire apartment in Nashville,SUPERHOST,/rooms/29543248?check_in=2020-12-17&check_out=...,4.93,76,6,2,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,$63,63,302,189,113
3,Tiny haven,Entire apartment in Nashville,SUPERHOST,/rooms/20137844?check_in=2020-12-17&check_out=...,4.93,193,4,4,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,$54,54,259,162,97
4,1 BR Apartment!! Just 2 miles to Downtown!,Entire apartment in Nashville,NaN,/rooms/44064578?check_in=2020-12-17&check_out=...,4.0,10,4,1,1 bedroom,1 bath,2020-12-17,2020-12-20,87,62,,62,257,186,71


In [298]:
weekends['not_discounted'] = weekends['not_discounted'].astype(str)

weekends['not_discounted'] = weekends['not_discounted'].str.replace('$','')
weekends['not_discounted'] = weekends['not_discounted'].str.replace(',','')
weekends['not_discounted'] = weekends['not_discounted'].str.replace(' total','')
weekends['not_discounted'] = weekends['not_discounted'].str.replace('nan', '0')

weekends.head()

,name,category,superhost,url,avg_stars,review_count,guests,beds,bedroom,bath,checkin,checkout,previous_price,discounted,not_discounted,current_nightly,total cost,room_fee,taxes_fees_cleaning
0,NC5 The Joint!! Downtown Nashville Studio!,Entire apartment in Nashville,NaN,/rooms/46779266?check_in=2020-12-17&check_out=...,,,6,2,Studio,1 bath,2020-12-17,2020-12-20,,,55,55,245,165,80
1,"Brand-New Condo, 2 Blocks to Centennial Park",Entire apartment in Nashville,NaN,/rooms/35440277?check_in=2020-12-17&check_out=...,4.64,14,4,2,1 bedroom,1 bath,2020-12-17,2020-12-20,122,58,,58,329,174,155
2,DOWNTOWN NASHVILLE - COMFY & COZY (C),Entire apartment in Nashville,SUPERHOST,/rooms/29543248?check_in=2020-12-17&check_out=...,4.93,76,6,2,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,63,63,302,189,113
3,Tiny haven,Entire apartment in Nashville,SUPERHOST,/rooms/20137844?check_in=2020-12-17&check_out=...,4.93,193,4,4,2 bedrooms,1 bath,2020-12-17,2020-12-20,,,54,54,259,162,97
4,1 BR Apartment!! Just 2 miles to Downtown!,Entire apartment in Nashville,NaN,/rooms/44064578?check_in=2020-12-17&check_out=...,4.0,10,4,1,1 bedroom,1 bath,2020-12-17,2020-12-20,87,62,,62,257,186,71
